In [1]:
import cv2
import numpy as np
import librosa
import sounddevice as sd
import matplotlib.pyplot as plt
import time
from transformers import pipeline
import atexit
import sys


c:\Users\Sandeep\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Sandeep\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Text Emotion Analysis

In [2]:
text_model = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
revision = "714eb0f"
text_analyzer = pipeline("sentiment-analysis", model=text_model, revision=revision)


c:\Users\Sandeep\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
def analyze_text(text):
    result = text_analyzer(text)
    return result[0]['label'], result[0]['score']
    

## Facial Expression Analysis

In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [5]:
def analyze_face(frame):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    emotions = ["Neutral", "Happy", "Sad", "Angry", "Surprised"]

    for (x, y, w, h) in faces:

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        emotion = np.random.choice(emotions)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        return emotion
        
    return "No Face Detected"


## Speech Emotion Detection

In [6]:
def analyze_speech(duration=3, sampling_rate=16000):

    print("Recording... Speak now!")

    audio = sd.rec(int(duration * sampling_rate), samplerate=sampling_rate, channels=1, dtype='float32')
    sd.wait()

    print("Recording complete!")

    mfccs = librosa.feature.mfcc(y=audio.flatten(), sr=sampling_rate, n_mfcc=40)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfccs, x_axis='time')
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()
    plt.show()

    emotion = np.random.choice(["Neutral", "Happy", "Sad", "Angry", "Fearful"])
    return emotion


## Real Time Emotion Detection

In [7]:
def real_time_emotion_detection():

    print("Launching Real-Time Emotion Detection...")

    cap = cv2.VideoCapture(0)
    start_time = time.time()
    detected_emotions = []

    try:
        while time.time() - start_time <= 60:

            ret, frame = cap.read()

            if not ret:
                break

            emotion_face = analyze_face(frame)

            if emotion_face != "No Face Detected":
                detected_emotions.append(emotion_face)

            cv2.imshow("Emotion Detection", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

        final_facial_emotion = max(set(detected_emotions), key=detected_emotions.count, default="No Face Detected")
        print(f"Final Facial Emotion: {final_facial_emotion}")

        text_input = input("Enter text to analyze emotion: ")
        emotion_text, confidence_text = analyze_text(text_input)
        print(f"Text Emotion: {emotion_text} (Confidence: {confidence_text:.2f})")

        emotion_speech = analyze_speech()
        print(f"Speech Emotion: {emotion_speech}")

    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        cleanup()

In [8]:
def cleanup():
    print("Cleaning up resources...")
    cv2.destroyAllWindows()
    sd.stop()


In [9]:
atexit.register(cleanup)


<function __main__.cleanup()>

In [11]:
if __name__ == "__main__":
    try:
        real_time_emotion_detection()
    except Exception as e:
        print(f"An error occurred: {e}")
    sys.exit(1)

Launching Real-Time Emotion Detection...
Final Facial Emotion: No Face Detected
Cleaning up resources...


KeyboardInterrupt: 